In [18]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from tqdm import tqdm
import numpy as np
from torchvision.datasets import CIFAR100
from torchvision.transforms import transforms
from ultralytics import YOLO

In [17]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version (PyTorch):", torch.version.cuda)
print("cuDNN Version:", torch.backends.cudnn.version())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


PyTorch Version: 2.5.1+cu124
CUDA Available: True
CUDA Version (PyTorch): 12.4
cuDNN Version: 90100
GPU Name: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [19]:
import os
from torchvision.datasets import CIFAR100
from torchvision.transforms import transforms
from PIL import Image

def prepare_cifar100_dataset(root, output_dir, train=True):
    """
    Prepare CIFAR-100 dataset for YOLO in hidden-object-detection format.
    """
    os.makedirs(output_dir, exist_ok=True)
    images_dir = os.path.join(output_dir, 'images')
    labels_dir = os.path.join(output_dir, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    # Load CIFAR-100 dataset
    dataset = CIFAR100(root=root, train=train, download=True)
    transform = transforms.Compose([transforms.Resize((640, 640))])

    for idx, (image, label) in enumerate(dataset):
        # Resize image
        image = transform(image)
        image_path = os.path.join(images_dir, f"{idx}.jpg")
        image.save(image_path)

        # Generate YOLO label
        x_center, y_center = 0.5, 0.5  # Normalized center
        box_width, box_height = 1.0, 1.0  # Normalized dimensions
        label_path = os.path.join(labels_dir, f"{idx}.txt")
        with open(label_path, "w") as f:
            f.write(f"{label} {x_center} {y_center} {box_width} {box_height}\n")

    print(f"Dataset prepared in YOLO format at {output_dir}.")


In [20]:
# Step 2: Prepare YOLO Training Configuration
def prepare_yolo_config(output_dir):
    """
    Generates a YAML file for YOLO training configuration.
    """
    yaml_content = f"""
    path: {output_dir}
    train: images
    val: images
    nc: 100
    names: [class_0, class_1, class_2, ..., class_99]  # Replace with actual class names
    """
    config_path = os.path.join(output_dir, "cifar100.yaml")
    with open(config_path, "w") as f:
        f.write(yaml_content)
    print(f"YOLO config saved to: {config_path}")
    return config_path

In [23]:
from ultralytics import YOLO
import os

# Step 1: Prepare CIFAR-100 in YOLO Format
def prepare_cifar100_yolo_format(data_dir, output_dir, num_images_per_class=10000):
    """
    Prepares the CIFAR-100 dataset for YOLO in hidden-object-detection format.
    """
    os.makedirs(output_dir, exist_ok=True)
    images_dir = os.path.join(output_dir, 'images')
    labels_dir = os.path.join(output_dir, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    from torchvision.datasets import CIFAR100
    from torchvision.transforms import transforms
    from PIL import Image

    # Load CIFAR-100
    dataset = CIFAR100(root=data_dir, train=True, download=True)
    transform = transforms.Compose([transforms.Resize((640, 640))])  # Resize for YOLO compatibility
    class_counts = [0] * 100

    for idx, (image, label) in enumerate(dataset):
        if class_counts[label] >= num_images_per_class:
            continue

        # Save image
        image = transform(image)
        image_path = os.path.join(images_dir, f"{idx}.jpg")
        image.save(image_path)

        # Save label
        label_path = os.path.join(labels_dir, f"{idx}.txt")
        with open(label_path, "w") as f:
            x_center, y_center, width, height = 0.5, 0.5, 1.0, 1.0  # Normalized box for the whole image
            f.write(f"{label} {x_center} {y_center} {width} {height}\n")

        class_counts[label] += 1
        if all(count >= num_images_per_class for count in class_counts):
            break

    print(f"Dataset prepared in YOLO format at: {output_dir}")

In [ ]:
def prepare_yolo_config(output_dir):
    """
    Creates a YAML config file for YOLO training.
    """
    # CIFAR-100 class names as placeholders (replace with actual class names if needed)
    class_names = [f"class_{i}" for i in range(100)]

    yaml_content = f"""
    path: {output_dir}
    train: images
    val: images
    nc: 100
    names: {class_names}
    """
    config_path = os.path.join(output_dir, "cifar100.yaml")
    with open(config_path, "w") as f:
        f.write(yaml_content)
    print(f"YOLO config saved to: {config_path}")
    return config_path


In [27]:
# Step 3: Train YOLOv8
def train_yolov8(config_path, model_checkpoint=None):
    """
    Trains YOLOv8 on the prepared CIFAR-100 dataset.
    """
    model = YOLO("yolov8n.pt")  # Load the YOLOv8 nano model (pretrained)
    model.train(
        data=config_path,
        epochs=100,
        batch=32,
        imgsz=640,
        device=0,  # Use GPU if available
        resume=model_checkpoint
    )
    print("Training complete.")

# Paths
data_dir = "./data"
output_dir = "./cifar100_yolo"
config_path = os.path.join(output_dir, "cifar100.yaml")

# Run the process
prepare_cifar100_yolo_format(data_dir, output_dir, num_images_per_class=10000)
config_path = prepare_yolo_config(output_dir)
train_yolov8(config_path)


Files already downloaded and verified
Dataset prepared in YOLO format at: ./cifar100_yolo
YOLO config saved to: ./cifar100_yolo/cifar100.yaml


100%|██████████| 6.25M/6.25M [00:17<00:00, 382kB/s]


Ultralytics 8.3.34 🚀 Python-3.9.19 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 5931MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./cifar100_yolo/cifar100.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=None, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

RuntimeError: Dataset 'cifar100_yolo/cifar100.yaml' error ❌ ./cifar100_yolo/cifar100.yaml 'names' length 4 and 'nc: 100' must match.

In [ ]:
# from ultralytics import YOLO
# model = YOLO('yolov8m.pt')


# results = model.train(data='CIFAR-100', epochs=50, imgsz=32)

In [ ]:
# # Ultralytics YOLO 🚀, AGPL-3.0 license
# # Objects365 dataset https://www.objects365.org/ by Megvii
# # Documentation: https://docs.ultralytics.com/datasets/detect/objects365/
# # Example usage: yolo train data=Objects365.yaml
# # parent
# # ├── ultralytics
# # └── datasets
# #     └── Objects365  ← downloads here (712 GB = 367G data + 345G zips)

# # Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
# path: ../datasets/Objects365 # dataset root dir
# train: images/train # train images (relative to 'path') 1742289 images
# val: images/val # val images (relative to 'path') 80000 images
# test: # test images (optional)

# # Classes
# names:
#   0: Person
#   1: Sneakers
#   2: Chair
#   3: Other Shoes
#   4: Hat
#   5: Car
#   6: Lamp
#   7: Glasses
#   8: Bottle
#   9: Desk
#   10: Cup
#   11: Street Lights
#   12: Cabinet/shelf
#   13: Handbag/Satchel
#   14: Bracelet
#   15: Plate
#   16: Picture/Frame
#   17: Helmet
#   18: Book
#   19: Gloves
#   20: Storage box
#   21: Boat
#   22: Leather Shoes
#   23: Flower
#   24: Bench
#   25: Potted Plant
#   26: Bowl/Basin
#   27: Flag
#   28: Pillow
#   29: Boots
#   30: Vase
#   31: Microphone
#   32: Necklace
#   33: Ring
#   34: SUV
#   35: Wine Glass
#   36: Belt
#   37: Monitor/TV
#   38: Backpack
#   39: Umbrella
#   40: Traffic Light
#   41: Speaker
#   42: Watch
#   43: Tie
#   44: Trash bin Can
#   45: Slippers
#   46: Bicycle
#   47: Stool
#   48: Barrel/bucket
#   49: Van
#   50: Couch
#   51: Sandals
#   52: Basket
#   53: Drum
#   54: Pen/Pencil
#   55: Bus
#   56: Wild Bird
#   57: High Heels
#   58: Motorcycle
#   59: Guitar
#   60: Carpet
#   61: Cell Phone
#   62: Bread
#   63: Camera
#   64: Canned
#   65: Truck
#   66: Traffic cone
#   67: Cymbal
#   68: Lifesaver
#   69: Towel
#   70: Stuffed Toy
#   71: Candle
#   72: Sailboat
#   73: Laptop
#   74: Awning
#   75: Bed
#   76: Faucet
#   77: Tent
#   78: Horse
#   79: Mirror
#   80: Power outlet
#   81: Sink
#   82: Apple
#   83: Air Conditioner
#   84: Knife
#   85: Hockey Stick
#   86: Paddle
#   87: Pickup Truck
#   88: Fork
#   89: Traffic Sign
#   90: Balloon
#   91: Tripod
#   92: Dog
#   93: Spoon
#   94: Clock
#   95: Pot
#   96: Cow
#   97: Cake
#   98: Dining Table
#   99: Sheep
#   100: Hanger
#   101: Blackboard/Whiteboard
#   102: Napkin
#   103: Other Fish
#   104: Orange/Tangerine
#   105: Toiletry
#   106: Keyboard
#   107: Tomato
#   108: Lantern
#   109: Machinery Vehicle
#   110: Fan
#   111: Green Vegetables
#   112: Banana
#   113: Baseball Glove
#   114: Airplane
#   115: Mouse
#   116: Train
#   117: Pumpkin
#   118: Soccer
#   119: Skiboard
#   120: Luggage
#   121: Nightstand
#   122: Tea pot
#   123: Telephone
#   124: Trolley
#   125: Head Phone
#   126: Sports Car
#   127: Stop Sign
#   128: Dessert
#   129: Scooter
#   130: Stroller
#   131: Crane
#   132: Remote
#   133: Refrigerator
#   134: Oven
#   135: Lemon
#   136: Duck
#   137: Baseball Bat
#   138: Surveillance Camera
#   139: Cat
#   140: Jug
#   141: Broccoli
#   142: Piano
#   143: Pizza
#   144: Elephant
#   145: Skateboard
#   146: Surfboard
#   147: Gun
#   148: Skating and Skiing shoes
#   149: Gas stove
#   150: Donut
#   151: Bow Tie
#   152: Carrot
#   153: Toilet
#   154: Kite
#   155: Strawberry
#   156: Other Balls
#   157: Shovel
#   158: Pepper
#   159: Computer Box
#   160: Toilet Paper
#   161: Cleaning Products
#   162: Chopsticks
#   163: Microwave
#   164: Pigeon
#   165: Baseball
#   166: Cutting/chopping Board
#   167: Coffee Table
#   168: Side Table
#   169: Scissors
#   170: Marker
#   171: Pie
#   172: Ladder
#   173: Snowboard
#   174: Cookies
#   175: Radiator
#   176: Fire Hydrant
#   177: Basketball
#   178: Zebra
#   179: Grape
#   180: Giraffe
#   181: Potato
#   182: Sausage
#   183: Tricycle
#   184: Violin
#   185: Egg
#   186: Fire Extinguisher
#   187: Candy
#   188: Fire Truck
#   189: Billiards
#   190: Converter
#   191: Bathtub
#   192: Wheelchair
#   193: Golf Club
#   194: Briefcase
#   195: Cucumber
#   196: Cigar/Cigarette
#   197: Paint Brush
#   198: Pear
#   199: Heavy Truck
#   200: Hamburger
#   201: Extractor
#   202: Extension Cord
#   203: Tong
#   204: Tennis Racket
#   205: Folder
#   206: American Football
#   207: earphone
#   208: Mask
#   209: Kettle
#   210: Tennis
#   211: Ship
#   212: Swing
#   213: Coffee Machine
#   214: Slide
#   215: Carriage
#   216: Onion
#   217: Green beans
#   218: Projector
#   219: Frisbee
#   220: Washing Machine/Drying Machine
#   221: Chicken
#   222: Printer
#   223: Watermelon
#   224: Saxophone
#   225: Tissue
#   226: Toothbrush
#   227: Ice cream
#   228: Hot-air balloon
#   229: Cello
#   230: French Fries
#   231: Scale
#   232: Trophy
#   233: Cabbage
#   234: Hot dog
#   235: Blender
#   236: Peach
#   237: Rice
#   238: Wallet/Purse
#   239: Volleyball
#   240: Deer
#   241: Goose
#   242: Tape
#   243: Tablet
#   244: Cosmetics
#   245: Trumpet
#   246: Pineapple
#   247: Golf Ball
#   248: Ambulance
#   249: Parking meter
#   250: Mango
#   251: Key
#   252: Hurdle
#   253: Fishing Rod
#   254: Medal
#   255: Flute
#   256: Brush
#   257: Penguin
#   258: Megaphone
#   259: Corn
#   260: Lettuce
#   261: Garlic
#   262: Swan
#   263: Helicopter
#   264: Green Onion
#   265: Sandwich
#   266: Nuts
#   267: Speed Limit Sign
#   268: Induction Cooker
#   269: Broom
#   270: Trombone
#   271: Plum
#   272: Rickshaw
#   273: Goldfish
#   274: Kiwi fruit
#   275: Router/modem
#   276: Poker Card
#   277: Toaster
#   278: Shrimp
#   279: Sushi
#   280: Cheese
#   281: Notepaper
#   282: Cherry
#   283: Pliers
#   284: CD
#   285: Pasta
#   286: Hammer
#   287: Cue
#   288: Avocado
#   289: Hami melon
#   290: Flask
#   291: Mushroom
#   292: Screwdriver
#   293: Soap
#   294: Recorder
#   295: Bear
#   296: Eggplant
#   297: Board Eraser
#   298: Coconut
#   299: Tape Measure/Ruler
#   300: Pig
#   301: Showerhead
#   302: Globe
#   303: Chips
#   304: Steak
#   305: Crosswalk Sign
#   306: Stapler
#   307: Camel
#   308: Formula 1
#   309: Pomegranate
#   310: Dishwasher
#   311: Crab
#   312: Hoverboard
#   313: Meatball
#   314: Rice Cooker
#   315: Tuba
#   316: Calculator
#   317: Papaya
#   318: Antelope
#   319: Parrot
#   320: Seal
#   321: Butterfly
#   322: Dumbbell
#   323: Donkey
#   324: Lion
#   325: Urinal
#   326: Dolphin
#   327: Electric Drill
#   328: Hair Dryer
#   329: Egg tart
#   330: Jellyfish
#   331: Treadmill
#   332: Lighter
#   333: Grapefruit
#   334: Game board
#   335: Mop
#   336: Radish
#   337: Baozi
#   338: Target
#   339: French
#   340: Spring Rolls
#   341: Monkey
#   342: Rabbit
#   343: Pencil Case
#   344: Yak
#   345: Red Cabbage
#   346: Binoculars
#   347: Asparagus
#   348: Barbell
#   349: Scallop
#   350: Noddles
#   351: Comb
#   352: Dumpling
#   353: Oyster
#   354: Table Tennis paddle
#   355: Cosmetics Brush/Eyeliner Pencil
#   356: Chainsaw
#   357: Eraser
#   358: Lobster
#   359: Durian
#   360: Okra
#   361: Lipstick
#   362: Cosmetics Mirror
#   363: Curling
#   364: Table Tennis



In [ ]:
# from ultralytics import YOLO

# # Load a model
# model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)

# # Train the model
# results = model.train(data="Objects365.yaml", epochs=50, imgsz=640)

  0%|          | 5.62M/1.24G [03:25<12:52:42, 28.7kB/s]


KeyboardInterrupt: 